# Interpreting Decision Tree for Sentiment Classification Using Important Features





In [ ]:
import datascience
import numpy as np
import graphviz

from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 


from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.base import ClassifierMixin
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

from graphviz import *

import pandas as pd
import numpy as np

import joblib

#import os
# os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/My\ Drive/DS200Labs/Project/

Tree


In [ ]:
!ls /content/drive/My\ Drive/DS200Labs/Project/Tree/

In [ ]:
!ls /content/drive/My\ Drive/DS200Labs/

baby.csv				  IMDB5000movies.csv
BCdecisionTree.pdf			  IMDBReviewsSentiment.csv
Ben-NE-9-10-2015-pass-6.csv		  IRIS.csv
breast-cancer-wisconsin-data-windCSV.csv  player_data.csv
BreatCancerFinal.csv			  Project
DecisionTree.pdf			  salary_data.csv
faithful.csv				  SFSalaries2011_2014.csv


read the IMDBReviewsSentiment.csv file

In [ ]:
path= "/content/drive/My Drive/DS200Labs/IMDBReviewsSentiment.csv"
reviews = Table.read_table(path)
reviews.show(10)

review,sentiment
One of the other reviewers has mentioned that after watc ...,positive
A wonderful little production. The filming t ...,positive
I thought this was a wonderful way to spend time on a to ...,positive
Basically there's a family where a little boy (Jake) thi ...,negative
"Petter Mattei's ""Love in the Time of Money"" is a visuall ...",positive
"Probably my all-time favorite movie, a story of selfless ...",positive
I sure would like to see a resurrection of a up dated Se ...,positive
"This show was an amazing, fresh & innovative idea in the ...",negative
Encouraged by the positive comments about this film on h ...,negative
If you like original gut wrenching laughter you will lik ...,positive


Sample 1000 positive reviews

In [ ]:
pos_reviews = reviews.where("sentiment", are.equal_to('positive')).sample(1000, with_replacement = False)
pos_reviews.show(10)

review,sentiment
If you have not seen this excellent movie about life in ...,positive
Outside the household is a different world and the famil ...,positive
Greatly enjoyed this 1945 mystery thriller film about a ...,positive
"John Cassavetes' ""Opening Night"" is fantastic and fascin ...",positive
Heartland was in production about the same time as Micha ...,positive
Saw the movie last night w/o knowing anything about it ( ...,positive
I absolutely fell in love with this girls. let me tell y ...,positive
Basic meaning of the story is a reality. Cruel true real ...,positive
I guess I'm part of the silent minority who enjoyed this ...,positive
"Having seen the uncut version, I thought this film was b ...",positive


In [ ]:
pos_reviews2 = pos_reviews.with_column("classification", 1)
pos_reviews2.show(5)

review,sentiment,classification
If you have not seen this excellent movie about life in ...,positive,1
Outside the household is a different world and the famil ...,positive,1
Greatly enjoyed this 1945 mystery thriller film about a ...,positive,1
"John Cassavetes' ""Opening Night"" is fantastic and fascin ...",positive,1
Heartland was in production about the same time as Micha ...,positive,1


Sample 1000 negative reviews

In [ ]:
neg_reviews = reviews.where("sentiment", are.equal_to('negative')).sample(1000, with_replacement=False)
neg_reviews.show(10)

review,sentiment
I ran across Yvette McClendon in a film at the Los Angel ...,negative
Lolita is a rebel and she's going to share to our wide o ...,negative
If you ever plan on renting (hopefully not buying) this ...,negative
When I saw that IMDb users rated this movie the bottom 2 ...,negative
"I purchased this film on DVD for £4, but it was a waste, ...",negative
"Unspeakably discombobulated turkey, a mix of anti-Nazi m ...",negative
"This film promised a lot, so many beautiful and well pla ...",negative
I couldn't believe my eyes once I've watched this movie. ...,negative
My mom would not let me watch this film when I was in gr ...,negative
I am a pretty much a sucker for those Ghost Hunter shows ...,negative


In [ ]:
neg_reviews2 = neg_reviews.with_column("classification", 0)
neg_reviews2.show(5)

review,sentiment,classification
I ran across Yvette McClendon in a film at the Los Angel ...,negative,0
Lolita is a rebel and she's going to share to our wide o ...,negative,0
If you ever plan on renting (hopefully not buying) this ...,negative,0
When I saw that IMDb users rated this movie the bottom 2 ...,negative,0
"I purchased this film on DVD for £4, but it was a waste, ...",negative,0


In [ ]:
reviews2 = pos_reviews2.append(neg_reviews2)
reviews2.show(5)
data = reviews2

review,sentiment,classification
If you have not seen this excellent movie about life in ...,positive,1
Outside the household is a different world and the famil ...,positive,1
Greatly enjoyed this 1945 mystery thriller film about a ...,positive,1
"John Cassavetes' ""Opening Night"" is fantastic and fascin ...",positive,1
Heartland was in production about the same time as Micha ...,positive,1


extract the text of reviews into variable data_X, classification of review into variable data_Y, and calculate the number of positive reviews and negative reviews.

In [ ]:
# Use the data for constructing a Decision Tree Classifier for Review Sentiment
data_X= list(data['review'])
data_Y= list(data['classification'])
pos_count = data.where('classification', are.equal_to(1)).num_rows
neg_count = data.where('classification', are.equal_to(0)).num_rows

print('Data size:', len(data_X))
print('Positive Data Size: ', pos_count)
print('Negative Data Size: ', neg_count)

Data size: 2000
Positive Data Size:  1000
Negative Data Size:  1000


### Stratified KFold Cross Validation



In [ ]:
# Use Stratified Kfold Cross Validation so that
#   each fold contains the same ratio of positive/negative instances

k = 10
skf = StratifiedKFold(n_splits= k , random_state=1, shuffle= True)

In [ ]:
### Set the range of max_depth to be the best_max_depth
Best_max_depth = 5
low_max_depth = Best_max_depth 
high_max_depth = Best_max_depth 
training_performance_table = np.empty( [high_max_depth+1, k+1, 5] )
testing_performance_table = np.empty( [high_max_depth+1, k+1, 5] )

### Create a CountVectorizer to map word features of all words in labelled data to index of word frequency matrix
### Note that this does NOT mean all tweets are used for training, it only means we create the columns of
### word frequency matrix using all words in the labelled tweets.  
count_vect = CountVectorizer(token_pattern='(?:([@#]|[0-9]|[a-z]|[A-Z])+)', analyzer= 'word', min_df=2)
x_all = list(data['review'])
count_vect = count_vect.fit(x_all)


# Creating Decision Tree model each fold



In [ ]:
k = 10
skf = StratifiedKFold(n_splits=k, random_state=1, shuffle= True)

Best_max_depth = 5
low_max_depth = Best_max_depth
high_max_depth = Best_max_depth 
training_performance_table= np.empty( [high_max_depth +1, k+1, 5] )
testing_performance_table = np.empty( [high_max_depth+1, k+1, 5] )

for depth in range(low_max_depth, high_max_depth+1):
    
    ### Create a CountVectorizer to map word features of all words in labelled reviews to index of a word frequency matrix.
    count_vect = CountVectorizer(stop_words="english", analyzer='word', ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)
    x_all = list(data['review'])
    ### The fit method only creates the column index for the Word Frequency matrix for all words in the review.
    ### Because we did not use fit_transform, no word frequency is filled in yet.
    empty_WF_matrix = count_vect.fit(x_all)
    
    ### Prepare the feature_importance_array to store all feature importance measures for all k folds
    feature_names = count_vect.get_feature_names_out()
    feature_names_array = np.asarray(feature_names)
    WF_dimension = len(feature_names)
    feature_importance_array = np.empty([k+1, WF_dimension])
    #Extract only words (not its first character) from feature_names into a list of words only
    words_only_list = list() 
    for i in range(0, WF_dimension):
      words_only_list.append(feature_names_array[i])
    print("Words_only_list:")
    print(words_only_list)
    
    # Initialize the fold to 1.  The value of fold will increase by 1 in each iteration of the loop below.
    fold = 1
    print("***** Maximum Depth of Decision Tree is ", depth, "******")
    for train_index, test_index in skf.split(data_X, data_Y):
        print("Fold Number:", fold)
        #   print("Training Data Index:", train_index)
        #   print("Testing Data Index:", test_index)  
        x_train= list(data.take(train_index)['review'])     
        #    print("Training Data:", x_train)
        y_train= list(data.take(train_index)['classification'])
        #    print("Training Data Target Output:", y_train)
        x_test= list(data.take(test_index)['review'])
        #    print("Testing Data:", x_test)
        y_test= list(data.take(test_index)['classification'])
        #   print("Testing Data Target Output:", y_test)
    
        #   We use transform here, rather than fit_transform, because the word frequency columns have been identified 
        #   earlier using fit
        X_train_word_vect = count_vect.transform(x_train)
        clf = tree.DecisionTreeClassifier(criterion='entropy', random_state = 100, max_depth=depth, \
                                  min_samples_leaf =2)
        clf.fit(X_train_word_vect, y_train)
        ### Use the model generated to predict for training data
        predicted_training_y = clf.predict(X_train_word_vect)
        train_p = metrics.precision_score(y_train, predicted_training_y)
        train_r = metrics.recall_score(y_train, predicted_training_y)
        train_f1= metrics.f1_score(y_train, predicted_training_y)
        training_performance_table[depth, fold, 0]=depth
        training_performance_table[depth, fold, 1]=train_p
        training_performance_table[depth, fold, 2]=train_r
        training_performance_table[depth, fold, 3]=train_f1
        print("Max depth is ", depth, "Prediction Performance for Training Data f1:", train_f1)
        
        ### Use the model generated to predict for testing data
        X_test_word_vect = count_vect.transform(x_test)
        predicted_testing_y = clf.predict(X_test_word_vect)
        
        test_p= metrics.precision_score(y_test, predicted_testing_y)
        test_r= metrics.recall_score(y_test, predicted_testing_y)
        test_f1 = metrics.f1_score(y_test, predicted_testing_y)
        testing_performance_table[depth, fold, 0]=depth
        testing_performance_table[depth, fold, 1]=test_p
        testing_performance_table[depth, fold, 2]= test_r
        testing_performance_table[depth, fold, 3]= test_f1
        #   print("Prediction Performance for Testing Data f1:", test_f1)  
    
        ### Obtain feature importance of the model for all words using .feature_importances_   
        feature_imp = clf.feature_importances_
        ### We store feature importance of kth fold in fold-i row of feature_importance_array
        ### because the first row of the array has index 0.
        feature_importance_array[fold-1] = feature_imp
        #   print("Feature_importance_array:")
        #   print(feature_importance_array)
        
        ### Combine feature importance of this model and feature names to find top 20 important features
        feature_imp_names = zip(feature_imp, words_only_list)
        top_20_features = sorted(feature_imp_names, reverse=True)[:21]
        print("Top 20 important features for fold ", fold)
        for importance, name in top_20_features:
            print(name, importance)
            
        ### Store a visualization of the decision tree model
        dot_data= tree.export_graphviz(clf, out_file=None, feature_names=count_vect.get_feature_names())
        graph = graphviz.Source(dot_data)
        #### Create a string (to be used as the name of the file for storing the deicion tree visualization)
        #### We include fold number in the file name so that we can compare trees with the feature importance of different folds.
        graph_file_path = "/content/drive/My Drive/DS200Labs/Project/Tree/DT_" + str(depth) + "_Fold_" + str(fold)
        graph.render(graph_file_path)
        
        fold=fold+1 
        ### End of for loop for k-fold Stratified Cross Validation

    ### Convert Feature Importance Array for all k folds to a Panda DataFrame
    ### with column headings being features (words_only)
    feature_importance_k_fold_DF = pd.DataFrame(data=feature_importance_array, columns=words_only_list)
    
    ### Calculate the mean of feature importance for all words across all k folds
    feature_imp_k_fold_avg = np.mean(feature_importance_array, axis=0)
    ### Add the average as the last row (row index k) to the feature_importance_k_fold_DF
    ### For example, for a 10 fold feature importance array, row 0 to row 9 stores feature
    ### importance measure for fold 1 to fold 10.  Hence, the average feature importance can 
    ### be stored in row 10.
    feature_importance_k_fold_DF.loc[k] = feature_imp_k_fold_avg
    
    ### Sort the feature_importance_k_fold_DF by row k (i.e., average of feature importance) in DESCENDING order.
    sorted_feature_importance_k_fold_DF=feature_importance_k_fold_DF.sort_values(by= k, axis='columns', ascending=False)
    print("Feature_importance_k_fold_DF sorted")
    print(sorted_feature_importance_k_fold_DF)
    ### Save the sorted feature importance DF as a CSV file 
    filename2 = "/content/drive/My Drive/DS200Labs/Project/DTcsvfile_" + str(depth) + "_for_" + str(k) + "_Fold" + ".csv"
    sorted_feature_importance_k_fold_DF.to_csv(filename2)       
    
    ### Print top 40 features with highest average feature importance across all folds
    feature_imp_avg_words = zip(feature_imp_k_fold_avg, words_only_list)
    top_40_K_fold_features = sorted(feature_imp_avg_words, reverse=True)[:41]
    print("Top 40 important features across all k folds")
    for importance, word in top_40_K_fold_features:
            print(word, importance)

# End of for loop for max_depth

Words_only_list:
['00', '000', '001', '007', '00s', '01', '02', '04', '05', '08', '09', '0f', '10', '100', '1000', '1000000', '100ft', '100th', '100x', '101', '102', '103', '107', '108', '10s', '10th', '11', '110', '1146', '117', '11th', '12', '120', '12a', '12m', '13', '135', '13th', '14', '140', '1479', '14th', '15', '150', '1500b', '150k', '16', '1602', '166', '16mm', '16th', '16ème', '17', '1740', '1794', '17th', '18', '180', '1800s', '1820', '1830', '1839', '1840', '1846', '1847', '1850', '1850s', '1857', '1859', '1860', '1861', '1863', '187', '1870', '1872', '1880', '1890s', '18th', '19', '1900', '1900s', '1902', '1903', '1904', '1905', '1906', '1907', '1910', '1913', '1915', '1916', '1920', '1920s', '1921', '1922', '1924', '1925', '1928', '1929', '1930', '1930s', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1940s', '1941', '1942', '1943', '1945', '1946', '1947', '1948', '1949', '1950', '1950s', '1951', '1952', '1953', '1954', '1955', '1956', '

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Fold Number: 2
Max depth is  5 Prediction Performance for Training Data f1: 0.7679057116953762
Top 20 important features for fold  2
bad 0.295499231002002
worst 0.14855394587170143
waste 0.10112622342952926
awful 0.08107271772193653
great 0.06655742549562343
excellent 0.0656137777982955
learn 0.03987610524909671
pull 0.032391371184489724
best 0.02845342847078905
lives 0.028289790675533145
stars 0.02479337943198296
fan 0.023834357126923094
way 0.023501634748997687
lot 0.018504606533189152
years 0.015255520935888481
don 0.006676484324021803
über 0.0
état 0.0
émigrés 0.0
zuniga 0.0
zudina 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Fold Number: 3
Max depth is  5 Prediction Performance for Training Data f1: 0.7598729005901045
Top 20 important features for fold  3
bad 0.23871486783838355
great 0.16073153799462936
worst 0.14847904401395906
waste 0.1287192331843986
awful 0.0898360475149467
famous 0.034801334630609665
boring 0.031977133008905506
saying 0.022804297915307147
best 0.021874208683550767
attempt 0.021585466550758964
guess 0.020394400540585163
lot 0.01938394734924644
lives 0.018608591001893195
fan 0.01669851241679545
pacing 0.013160825731700335
feels 0.012230551624330252
über 0.0
état 0.0
émigrés 0.0
zuniga 0.0
zudina 0.0


...

Fold Number: 9
Max depth is  5 Prediction Performance for Training Data f1: 0.7751865671641791
Top 20 important features for fold  9
bad 0.24759802853029492
worst 0.14114570104948237
waste 0.10326596169808645
awful 0.08306394331895267
great 0.08159198355998859
stupid 0.06648320709260366
best 0.05409283512807298
boring 0.028507584913885033
control 0.026685254537291238
lives 0.02607131559780039
looking 0.022636410914336227
unlike 0.02186695042623276
attempt 0.021764476948114904
lot 0.018587189893978756
fan 0.016179383747585863
ridiculous 0.016047041610504572
shot 0.012684903802677396
spent 0.01172782723011133
über 0.0
état 0.0
émigrés 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Fold Number: 10
Max depth is  5 Prediction Performance for Training Data f1: 0.7690229626294461
Top 20 important features for fold  10
bad 0.2526105618833276
worst 0.14837082652308184
great 0.12627435982884705
waste 0.10204012014389686
awful 0.0992501216261138
best 0.06403279267873176
boring 0.05091402849477125
lives 0.0279159594566462
just 0.024542603355047828
nice 0.02218498520584403
unlike 0.0216053876123319
attempt 0.018255899716851227
haven 0.012658233609146703
late 0.011587544134779894
short 0.011130512799239472
make 0.006626062931342656
über 0.0
état 0.0
émigrés 0.0
zuniga 0.0
zudina 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Feature_importance_k_fold_DF sorted
         bad     worst     great     waste  ...  rosen  towers  role  brignon
0   0.241281  0.129204  0.086211  0.141226  ...    0.0     0.0   0.0      0.0
1   0.295499  0.148554  0.066557  0.101126  ...    0.0     0.0   0.0      0.0
2   0.238715  0.148479  0.160732  0.128719  ...    0.0     0.0   0.0      0.0
3   0.249532  0.132870  0.144604  0.101711  ...    0.0     0.0   0.0      0.0
4   0.237200  0.142062  0.158985  0.100610  ...    0.0     0.0   0.0      0.0
5   0.269401  0.128131  0.144812  0.118309  ...    0.0     0.0   0.0      0.0
6   0.266904  0.145776  0.129360  0.101853  ...    0.0     0.0   0.0      0.0
7   0.252925  0.136069  0.151878  0.105766  ...    0.0     0.0   0.0      0.0
8   0.247598  0.141146  0.081592  0.103266  ...    0.0     0.0   0.0      0.0
9   0.252611  0.148371  0.126274  0.102040  ...    0.0     0.0   0.0      0.0
10  0.231970  0.127333  0.113728  0.100421  ...    0.0     0.0   0.0      0.0

[11 rows x 24800 columns]
T

Words like "bad", "worst", "great", "waste" are strong sentimental words that shows how you felt about the movie unlike "let", "feels", "knows"
Among the top 40 features, only 6 of them had all non-zero importance and 13 of them had non-zero importance at least in 5 folds
